# 03. Módulo de Verificación (Código Fuente Incluido)

Este notebook implementa el sistema de verificación GDPR basado en IA. Combina una base de datos de grafos (Neo4j) para el contexto legal y un LLM (Gemma) para el análisis de imágenes.

### 1. Configuración Inicial
Cargamos variables de entorno (`.env`) para conectar con Neo4j y definimos configuraciones básicas como el modelo de LLM que vamos a usar.

In [8]:
import os
import json
import asyncio
import logging
import time  # NUEVO: Para cache TTL
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)

# Cargar environment (ajusta la ruta si hace falta)
load_dotenv(os.path.abspath(os.path.join("..", ".env")))

# Configuración de cache
CACHE_TTL_SECONDS = 300  # 5 minutos

# Simulación de GDPRQueries y config
class GDPRQueries:
    GET_ARTICLES_FOR_DETECTION = "MATCH (a:GDPRArticle {category: $detection_type}) RETURN a.article_number as article_number, a.title as title, a.content as content"
    SEMANTIC_SEARCH = "" # Placeholder
    GET_FINE_INFO = ""
    GET_EXPLANATION_GRAPH = ""

def get_config():
    class Config:
        verification = {"llm_model": "google/gemma-3n-E4B-it"}
    return Config()

### 2. Cliente de Grafo (`graph_client.py`)
Maneja la conexión a la base de datos de grafos Neo4j.
- **`_init_driver`**: Conecta usando las credenciales del `.env`.
- **`get_context_for_detection`**: Dado un tipo de detección (ej. "persona"), busca en el grafo qué artículos del GDPR se aplican.
    - Si la DB no está disponible, devuelve un mock para que el notebook no falle.

In [9]:
from neo4j import GraphDatabase, AsyncGraphDatabase

class GraphClient:
    """
    Cliente de grafo mejorado con cache de contexto para evitar queries repetidas.
    """
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(GraphClient, cls).__new__(cls)
            cls._instance._init_driver()
            cls._instance._context_cache = {}  # NUEVO: Cache de contexto
            cls._instance._cache_timestamps = {}  # NUEVO: Timestamps para TTL
        return cls._instance
    
    def _init_driver(self):
        uri = os.getenv("NEO4J_URI", "bolt://localhost:7687")
        user = os.getenv("NEO4J_USER", "neo4j")
        password = os.getenv("NEO4J_PASSWORD", "password")
        try:
            self.driver = AsyncGraphDatabase.driver(uri, auth=(user, password))
        except Exception as e:
            print(f"Failed to init Neo4j driver: {e}")
            self.driver = None
        
    async def close(self):
        if self.driver:
            await self.driver.close()
    
    def _is_cache_valid(self, key: str) -> bool:
        """NUEVO: Verifica si el cache sigue siendo válido (TTL)"""
        if key not in self._cache_timestamps:
            return False
        return (time.time() - self._cache_timestamps[key]) < CACHE_TTL_SECONDS
    
    def clear_cache(self):
        """NUEVO: Limpia el cache manualmente si es necesario"""
        self._context_cache = {}
        self._cache_timestamps = {}
            
    async def get_context_for_detection(self, detection_type: str) -> List[Dict[str, Any]]:
        # NUEVO: Verificar cache primero
        cache_key = f"context:{detection_type}"
        if cache_key in self._context_cache and self._is_cache_valid(cache_key):
            return self._context_cache[cache_key]
        
        if not self.driver:
            # Fallback mock context if DB is not up
            result = [{
                "article_number": "6",
                "title": "Lawfulness of processing",
                "content": "Processing shall be lawful only if..."
            }]
            # Cache el mock también
            self._context_cache[cache_key] = result
            self._cache_timestamps[cache_key] = time.time()
            return result
            
        try:
            async with self.driver.session() as session:
                result = await session.run(
                    GDPRQueries.GET_ARTICLES_FOR_DETECTION,
                    {"detection_type": detection_type}
                )
                records = await result.data()
                
                # NUEVO: Guardar en cache
                self._context_cache[cache_key] = records
                self._cache_timestamps[cache_key] = time.time()
                
                return records
        except Exception as e:
            print(f"Neo4j Error: {e}")
            return []

    async def semantic_search(self, *args, **kwargs): 
        return []

### 3. Cliente LLM (`gemma_client.py`)
Interfaz con el modelo de lenguaje (Gemma).
- **`analyze_image`**: Recibiría la imagen y el contexto legal, y preguntaría al LLM si hay violación de privacidad.
- **`_mock_response`**: En este notebook usamos un mock para no depender de tener el modelo de 4GB cargado en memoria local, simulando que detecta una violación si es "face" o "license_plate".

In [10]:
# Severidad GDPR por tipo de dato
GDPR_SEVERITY_MAP = {
    "face": "high",           # Dato biométrico - Art. 9 GDPR
    "fingerprint": "high",    # Dato biométrico - Art. 9 GDPR
    "license_plate": "high",  # Identificador personal indirecto
    "id_document": "high",    # Documento oficial
    "credit_card": "high",    # Dato financiero
    "signature": "medium",    # Identificador personal
    "person": "medium",       # Imagen personal
}

# Artículos GDPR relacionados por tipo
GDPR_ARTICLES = {
    "face": ["9", "6"],           # Datos biométricos + Base legal
    "fingerprint": ["9", "6"],
    "license_plate": ["6", "17"], # Base legal + Derecho al olvido
    "person": ["6", "13"],        # Base legal + Información
    "id_document": ["9", "6", "32"],  # Especial + Seguridad
    "credit_card": ["6", "32"],   # Base legal + Seguridad
    "signature": ["6"],
}

# =============================================================================
# PROMPTS PARA CLASIFICACIÓN DE CONTENIDO SENSIBLE
# =============================================================================

SENSITIVE_CONTENT_CLASSIFICATION_PROMPT = """
Analyze this image crop and determine if it contains any of the following sensitive content:

1. **Fingerprint**: A finger showing visible ridge patterns (the lines/whorls on fingertips)
2. **ID Document**: Any official identification document (passport, national ID, driver's license, residence permit)
3. **Credit Card**: A bank card, credit card, or debit card showing numbers or chip
4. **Signature**: A handwritten signature or autograph
5. **Hand with biometric features**: A hand that could be used for palm print recognition

IMPORTANT RULES:
- A regular hand holding something is NOT a fingerprint unless ridge patterns are clearly visible
- A blank card or generic plastic is NOT a credit card unless it has numbers/chip visible
- Random text is NOT a signature unless it's clearly a handwritten personal signature

Respond with a JSON object:
{
  "detected_type": "fingerprint|id_document|credit_card|signature|hand_biometric|none",
  "confidence": 0.0 to 1.0,
  "reasoning": "Brief explanation of why this classification was made"
}

If no sensitive content is detected, return:
{
  "detected_type": "none",
  "confidence": 0.9,
  "reasoning": "No sensitive personal data detected in this crop"
}
"""

GDPR_VIOLATION_ANALYSIS_PROMPT = """
You are a GDPR compliance expert. Analyze this detection and determine if it constitutes a privacy violation.

Detection Type: {detection_type}
Legal Context (GDPR Articles):
{legal_context}

Based on GDPR regulations, determine:
1. Is this a privacy violation? (recording/processing without consent)
2. What is the severity? (high/medium/low)
3. Which specific GDPR articles apply?
4. What action should be taken? (blur/pixelate/redact)

Respond with JSON:
{{
  "is_violation": true/false,
  "severity": "high|medium|low|none",
  "violated_articles": ["6", "9", ...],
  "reasoning": "Explanation",
  "recommended_action": "blur|pixelate|redact|none",
  "confidence": 0.0 to 1.0
}}
"""

class GemmaClient:
    """
    Cliente LLM mejorado con:
    - Clasificación de contenido sensible (huellas, documentos, etc.)
    - Análisis de violaciones GDPR
    - Soporte para procesamiento de imágenes con visión
    """
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(GemmaClient, cls).__new__(cls)
        return cls._instance
    
    def __init__(self):
        if hasattr(self, 'config'):
            return
        self.config = get_config()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = None  # Se carga bajo demanda
        self.processor = None
        
        # Cache de clasificaciones para evitar re-procesar
        self._classification_cache = {}
    
    def _load_model(self):
        """Carga el modelo LLM bajo demanda"""
        if self.model is not None:
            return
        
        try:
            from transformers import AutoProcessor, AutoModelForCausalLM
            model_name = self.config.verification.llm_model
            
            logging.info(f"Loading LLM model: {model_name}")
            self.processor = AutoProcessor.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else None
            )
            logging.info("LLM model loaded successfully")
        except Exception as e:
            logging.warning(f"Could not load LLM model: {e}. Using mock responses.")
            self.model = "mock"
    
    async def classify_sensitive_content(
        self, 
        image_path: str,
        use_cache: bool = True
    ) -> Dict[str, Any]:
        """
        Clasifica si una imagen contiene contenido sensible no detectable por YOLO.
        
        Tipos detectables:
        - fingerprint: Huellas dactilares visibles
        - id_document: Documentos de identidad (DNI, pasaporte, etc.)
        - credit_card: Tarjetas de crédito/débito
        - signature: Firmas manuscritas
        - hand_biometric: Manos con características biométricas visibles
        
        Args:
            image_path: Ruta a la imagen o crop a analizar
            use_cache: Usar cache para evitar re-clasificar
            
        Returns:
            Dict con detected_type, confidence, reasoning
        """
        # Verificar cache
        if use_cache and image_path in self._classification_cache:
            return self._classification_cache[image_path]
        
        # Cargar modelo si es necesario
        if self.model is None:
            self._load_model()
        
        # Si el modelo es mock, usar heurísticas
        if self.model == "mock":
            result = self._classify_mock(image_path)
        else:
            result = await self._classify_with_llm(image_path)
        
        # Guardar en cache
        if use_cache:
            self._classification_cache[image_path] = result
        
        return result
    
    def _classify_mock(self, image_path: str) -> Dict[str, Any]:
        """
        Clasificación mock basada en nombre de archivo para testing.
        En producción, esto sería reemplazado por análisis LLM real.
        """
        path_lower = image_path.lower()
        
        if "finger" in path_lower or "huella" in path_lower:
            return {
                "detected_type": "fingerprint",
                "confidence": 0.85,
                "reasoning": "Mock: Filename suggests fingerprint content"
            }
        elif "dni" in path_lower or "passport" in path_lower or "documento" in path_lower:
            return {
                "detected_type": "id_document",
                "confidence": 0.85,
                "reasoning": "Mock: Filename suggests ID document"
            }
        elif "card" in path_lower or "tarjeta" in path_lower or "visa" in path_lower:
            return {
                "detected_type": "credit_card",
                "confidence": 0.85,
                "reasoning": "Mock: Filename suggests credit card"
            }
        elif "firma" in path_lower or "signature" in path_lower:
            return {
                "detected_type": "signature",
                "confidence": 0.80,
                "reasoning": "Mock: Filename suggests signature"
            }
        
        return {
            "detected_type": "none",
            "confidence": 0.7,
            "reasoning": "Mock: No sensitive content detected in filename"
        }
    
    async def _classify_with_llm(self, image_path: str) -> Dict[str, Any]:
        """
        Clasificación real usando modelo de visión-lenguaje.
        """
        try:
            from PIL import Image
            import json
            
            image = Image.open(image_path).convert("RGB")
            
            # Preparar inputs para el modelo
            inputs = self.processor(
                text=SENSITIVE_CONTENT_CLASSIFICATION_PROMPT,
                images=image,
                return_tensors="pt"
            ).to(self.device)
            
            # Generar respuesta
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=False
            )
            
            response = self.processor.decode(outputs[0], skip_special_tokens=True)
            
            # Parsear JSON de la respuesta
            json_start = response.find("{")
            json_end = response.rfind("}") + 1
            if json_start != -1 and json_end > json_start:
                return json.loads(response[json_start:json_end])
            
            return {"detected_type": "none", "confidence": 0.5, "reasoning": "Could not parse LLM response"}
            
        except Exception as e:
            logging.error(f"LLM classification error: {e}")
            return {"detected_type": "none", "confidence": 0.3, "reasoning": f"Error: {str(e)}"}

    async def analyze_image(
        self, 
        image_path: str, 
        context: List[Dict[str, Any]], 
        detection_type: str
    ) -> Dict[str, Any]:
        """
        Analiza una imagen para determinar si hay violación GDPR.
        Combina clasificación de contenido con análisis legal.
        """
        # Si el tipo es "unknown" o "hand", intentar clasificar con LLM
        if detection_type in ("unknown", "hand", "hand_crop"):
            classification = await self.classify_sensitive_content(image_path)
            if classification["detected_type"] != "none":
                detection_type = classification["detected_type"]
                logging.info(f"LLM classified {image_path} as {detection_type}")
        
        # Determinar violación basada en tipo
        return self._determine_violation(detection_type, context)

    def _determine_violation(self, detection_type: str, context: List[Dict]) -> Dict[str, Any]:
        """
        Determina si hay violación basándose en el tipo de detección.
        
        Lógica GDPR:
        - Caras, huellas, documentos de identidad -> SIEMPRE violación
        - Matrículas, tarjetas de crédito -> Violación
        - Personas -> Depende del contexto
        """
        
        # Tipos que SIEMPRE son violación bajo GDPR (Art. 9 - Datos especiales)
        always_violation = {"face", "fingerprint", "id_document", "credit_card", "hand_biometric"}
        
        # Tipos que son violación con alta probabilidad
        likely_violation = {"license_plate", "signature"}
        
        # Tipos que dependen del contexto
        context_dependent = {"person"}
        
        is_violation = False
        severity = "none"
        confidence = 0.0
        reasoning = ""
        recommended_action = "none"
        
        if detection_type in always_violation:
            is_violation = True
            severity = "high"
            confidence = 0.95
            
            if detection_type == "fingerprint":
                reasoning = "Huella dactilar detectada: Dato biométrico especial según Art. 9 GDPR. Requiere consentimiento explícito para su procesamiento."
                recommended_action = "pixelate"
            elif detection_type == "id_document":
                reasoning = "Documento de identidad detectado: Contiene datos personales identificativos. Procesamiento requiere base legal según Art. 6 GDPR."
                recommended_action = "blur"
            elif detection_type == "credit_card":
                reasoning = "Tarjeta bancaria detectada: Dato financiero sensible. Requiere medidas de seguridad según Art. 32 GDPR."
                recommended_action = "pixelate"
            elif detection_type == "hand_biometric":
                reasoning = "Mano con características biométricas: Puede usarse para identificación. Art. 9 GDPR aplica."
                recommended_action = "blur"
            else:  # face
                reasoning = f"Detección de {detection_type}: Dato biométrico según Art. 9 GDPR. Requiere consentimiento explícito."
                recommended_action = "blur"
            
        elif detection_type in likely_violation:
            is_violation = True
            severity = "high"
            confidence = 0.90
            
            if detection_type == "signature":
                reasoning = "Firma manuscrita detectada: Identificador personal único. Art. 6 GDPR requiere base legal."
                recommended_action = "blur"
            else:  # license_plate
                reasoning = f"Detección de {detection_type}: Identificador personal que permite identificación indirecta según Art. 4 GDPR."
                recommended_action = "pixelate"
            
        elif detection_type in context_dependent:
            is_violation = True
            severity = "medium"
            confidence = 0.75
            reasoning = f"Detección de {detection_type}: Imagen personal que requiere base legal según Art. 6 GDPR. En espacios públicos, puede haber excepciones."
            recommended_action = "blur"
        
        else:
            is_violation = False
            severity = "none"
            confidence = 0.5
            reasoning = f"Tipo '{detection_type}' no clasificado como dato sensible GDPR."
            recommended_action = "none"
        
        return {
            "is_violation": is_violation,
            "severity": severity,
            "violated_articles": GDPR_ARTICLES.get(detection_type, ["6"]),
            "reasoning": reasoning,
            "confidence": confidence,
            "detection_type": detection_type,
            "recommended_action": recommended_action
        }

# Importar torch para verificación de CUDA
import torch
logging.info(f"CUDA available: {torch.cuda.is_available()}")

INFO:root:CUDA available: True


### 4. Sistema Multi-Agente (`sub_agent.py`, `consensus_agent.py`)
Aquí estructuramos la decisión:
- **`SubAgent`**: Representa un experto individual. Usa el grafo para informarse y el LLM para decidir sobre UNA detección específica.
- **`ConsensusAgent`**: Recibe las opiniones de los sub-agentes (si tuviéramos varios analizando lo mismo) y toma la decisión final. También normaliza el formato de salida.

In [11]:
class SubAgent:
    def __init__(self, agent_id: str):
        self.agent_id = agent_id
        self.graph_client = GraphClient()
        self.gemma_client = GemmaClient()
        
    async def verify(self, image_path: str, detection: Dict[str, Any]) -> Dict[str, Any]:
        detection_type = detection.get("detection_type", "unknown")
        context = await self.graph_client.get_context_for_detection(detection_type)
        res = await self.gemma_client.analyze_image(image_path, context, detection_type)
        res["agent_id"] = self.agent_id
        res["detection_id"] = detection.get("id")
        return res

class ConsensusAgent:
    def aggregate(self, results: List[Dict[str, Any]]) -> Dict[str, Any]:
        if not results: return {}
        return self._validate(results[0])

    def _validate(self, result: Dict[str, Any]) -> Dict[str, Any]:
        defaults = {"is_violation": False, "severity": "none", "confidence": 0.0}
        for k, v in defaults.items():
            if k not in result: result[k] = v
        return result

### 5. Procesador Paralelo (`parallel_processor.py`)
Orquesta la ejecución masiva.
- **`process_batch`**: Recibe una imagen y MUCHAS detecciones (ej. 10 personas en una foto).
- Crea un `SubAgent` para cada detección.
- Usa `asyncio.gather` para ejecutarlos todos a la vez (paralelismo), acelerando mucho el análisis.

In [12]:
class ParallelProcessor:
    def __init__(self):
        self.consensus = ConsensusAgent()
        
    async def process_batch(self, image_path: str, detections: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        tasks = []
        for det in detections:
            tasks.append(self._process_single(image_path, det))
        return await asyncio.gather(*tasks)
        
    async def _process_single(self, image_path: str, detection: Dict[str, Any]) -> Dict[str, Any]:
        agent = SubAgent(f"agent-{detection.get('detection_type')}")
        res = await agent.verify(image_path, detection)
        return self.consensus.aggregate([res])

processor = ParallelProcessor()

async def verify_image_detections(image_path: str, detections: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    return await processor.process_batch(image_path, detections)

### 6. Ejecución de Prueba
Simulamos una lista de 2 detecciones y llamamos al orquestador para ver si detecta violaciones.

In [13]:
# =============================================================================
# PRUEBA DE VERIFICACIÓN GDPR
# =============================================================================

# Test con diferentes tipos de detección GDPR
detections_mock = [
    {"id": "1", "detection_type": "face"},           # Alta severidad - Biométrico
    {"id": "2", "detection_type": "license_plate"},  # Alta severidad - Identificador
    {"id": "3", "detection_type": "person"},         # Media severidad - Contexto
    {"id": "4", "detection_type": "fingerprint"},    # Alta severidad - Biométrico
    {"id": "5", "detection_type": "id_document"},    # Alta severidad - Documento oficial
    {"id": "6", "detection_type": "credit_card"},    # Alta severidad - Financiero
    {"id": "7", "detection_type": "signature"},      # Media severidad - Identificador
]

async def test_verification():
    print("=" * 70)
    print("TEST 1: Verificación de Detecciones Conocidas según GDPR")
    print("=" * 70)
    
    results = await verify_image_detections(
        "dummy_path.jpg",
        detections_mock
    )
    
    print("\nResultados de verificación:\n")
    for r in results:
        status = "⚠️ VIOLACIÓN" if r.get("is_violation") else "✅ OK"
        severity_icon = {"high": "🔴", "medium": "🟡", "low": "🟢", "none": "⚪"}.get(r.get("severity", "none"), "⚪")
        
        print(f"{status} {severity_icon}")
        print(f"  Tipo: {r.get('detection_type', 'unknown')}")
        print(f"  Severidad: {r.get('severity', 'none')}")
        print(f"  Confianza: {r.get('confidence', 0):.0%}")
        print(f"  Artículos GDPR: {', '.join(r.get('violated_articles', []))}")
        print(f"  Acción recomendada: {r.get('recommended_action', 'none')}")
        print(f"  Razonamiento: {r.get('reasoning', 'N/A')[:80]}...")
        print()

async def test_llm_classification():
    print("=" * 70)
    print("TEST 2: Clasificación LLM de Contenido Sensible")
    print("=" * 70)
    print("\nEste test simula la clasificación de crops de imagen por el LLM.")
    print("En producción, el LLM analizaría la imagen real.\n")
    
    # Simular rutas de archivos que el mock clasificará
    test_paths = [
        "captures/track_1/huella_001.jpg",      # → fingerprint
        "captures/track_2/dni_front.jpg",       # → id_document
        "captures/track_3/tarjeta_visa.jpg",    # → credit_card
        "captures/track_4/firma_contrato.jpg",  # → signature
        "captures/track_5/mano_normal.jpg",     # → none
    ]
    
    client = GemmaClient()
    
    for path in test_paths:
        result = await client.classify_sensitive_content(path)
        
        detected = result.get("detected_type", "none")
        conf = result.get("confidence", 0)
        reason = result.get("reasoning", "")
        
        icon = "🔍" if detected != "none" else "➖"
        print(f"{icon} {path}")
        print(f"    Tipo detectado: {detected}")
        print(f"    Confianza: {conf:.0%}")
        print(f"    Razón: {reason}")
        print()

async def test_hand_crop_flow():
    print("=" * 70)
    print("TEST 3: Flujo Completo - Hand Crop → LLM → GDPR Verification")
    print("=" * 70)
    print("\nEste test simula el flujo cuando YOLO detecta una mano y el")
    print("LLM debe clasificar si contiene huella dactilar.\n")
    
    # Simular detecciones de manos que el LLM debe reclasificar
    hand_detections = [
        {"id": "h1", "detection_type": "hand", "image_path": "crops/hand_huella_visible.jpg"},
        {"id": "h2", "detection_type": "hand_crop", "image_path": "crops/hand_normal.jpg"},
    ]
    
    client = GemmaClient()
    processor = ParallelProcessor()
    
    for det in hand_detections:
        # El flujo interno reclasifica si es "hand"
        result = await processor._process_single(det["image_path"], det)
        
        print(f"Detección original: {det['detection_type']}")
        print(f"  → Clasificación final: {result.get('detection_type')}")
        print(f"  → Violación: {'Sí' if result.get('is_violation') else 'No'}")
        print(f"  → Severidad: {result.get('severity')}")
        print(f"  → Acción: {result.get('recommended_action')}")
        print()

async def run_all_tests():
    await test_verification()
    print("\n")
    await test_llm_classification()
    print("\n")
    await test_hand_crop_flow()
    
    print("=" * 70)
    print("RESUMEN DE CAPACIDADES")
    print("=" * 70)
    print("""
Tipos de datos sensibles detectables:
┌─────────────────┬────────────┬─────────────┬─────────────────────────┐
│ Tipo            │ Severidad  │ Artículos   │ Método de Detección     │
├─────────────────┼────────────┼─────────────┼─────────────────────────┤
│ face            │ Alta       │ 9, 6        │ YOLO                    │
│ fingerprint     │ Alta       │ 9, 6        │ LLM (contexto de mano)  │
│ license_plate   │ Alta       │ 6, 17       │ YOLO                    │
│ id_document     │ Alta       │ 9, 6, 32    │ LLM                     │
│ credit_card     │ Alta       │ 6, 32       │ LLM                     │
│ signature       │ Media      │ 6           │ LLM                     │
│ person          │ Media      │ 6, 13       │ YOLO                    │
└─────────────────┴────────────┴─────────────┴─────────────────────────┘
    """)

asyncio.run(run_all_tests())

INFO:root:LLM classified crops/hand_huella_visible.jpg as fingerprint


TEST 1: Verificación de Detecciones Conocidas según GDPR
Neo4j Error: {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}
Neo4j Error: {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}
Neo4j Error: {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}
Neo4j Error: {code: Neo.ClientError.Security.AuthenticationRateLimit} {message: The client has provided incorrect authentication details too many times in a row.}
Neo4j Error: {code: Neo.ClientError.Security.AuthenticationRateLimit} {message: The client has provided incorrect authentication details too many times in a row.}
Neo4j Error: {code: Neo.ClientError.Security.AuthenticationRateLimit} {message: The client has provided incorrect authentication details too many times in a row.}
Neo4j Error: {code: Neo.ClientError.Security.AuthenticationRateLimit} 